In [3]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import numpy as np
import pandas as pd
import math
import glob
import time
import pickle

import steering
import speeding

import logging

In [16]:
PROJECT_ROOT = '../../../..'
LEARNED_DRIVER = 'default_miner/data'


COMMAND = 'steering'
# COMMAND = 'speeding'
logging.basicConfig(filename='logs/{}/training-{}.log'.format(COMMAND, time.time()),level=logging.DEBUG)

NUM_EPOCHS = 150

DIFFICULTY = ['easy', 'medium']
TRAINING_FILES = []
for d in DIFFICULTY:
    TRAINING_FILES.extend(glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, d, '*.csv'])))
TRAINING_DATA = {}
for FILE in TRAINING_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    TRAINING_DATA[FILE] = DF.values

VALIDATION_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, 'validation', '*.csv']))
VALIDATION_DATA = {}
for FILE in VALIDATION_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    VALIDATION_DATA[FILE] = DF.values
    
CUDA = torch.cuda.is_available()
if CUDA:
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor

In [11]:
def save_checkpoint(state, is_best, filepath='latest_checkpoint.tar'):
    torch.save(state, 'split_checkpoints/' + DIFFICULTY[-1] + '/' + COMMAND + '/' + filepath)
    if is_best:
        torch.save(state, 'split_checkpoints/' + DIFFICULTY[-1] + '/' + COMMAND + '/' + 'best_checkpoint.tar')

In [12]:
def train(training_data, model, criterion, optimzier):
    loss = 0
    model.train(mode=True)
    for key in training_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        optimizer.zero_grad()
        track_sequence = training_data[key]

        if COMMAND == 'steering':
            targets = track_sequence[:, 2:3]
        elif COMMAND == 'speeding':
            targets = track_sequence[:, 0:2]
        
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets)).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs),  requires_grad=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        track_loss.backward()
        optimizer.step()

        loss += track_loss.data[0]
    return loss

In [13]:
def validate(validation_data, model, criterion):
    loss = 0
    model.train(mode=False)
    for key in validation_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        track_sequence = validation_data[key]

        if COMMAND == 'steering':
            targets = track_sequence[:, 2:3]
        elif COMMAND == 'speeding':
            targets = track_sequence[:, 0:2]
            
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets), volatile=True).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs), volatile=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        loss += track_loss.data[0]
    return loss

In [17]:
checkpoint = torch.load('split_checkpoints/medium/{}/latest_checkpoint.tar'.format(COMMAND))
if COMMAND == 'steering':
    model = steering.Steering(steering.HYPERPARAMS.INPUT_SIZE,
                               steering.HYPERPARAMS.LSTM_HIDDEN_SIZE,
                               steering.HYPERPARAMS.HIDDEN_LAYER_SIZE,
                               steering.HYPERPARAMS.DROPOUT_PROB,
                               steering.HYPERPARAMS.NUM_LAYERS,
                               steering.HYPERPARAMS.TARGET_SIZE,
                               steering.HYPERPARAMS.BATCH_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=steering.HYPERPARAMS.LEARNING_RATE)
elif COMMAND == 'speeding':
    model = speeding.Speeding(speeding.HYPERPARAMS.INPUT_SIZE,
                               speeding.HYPERPARAMS.LSTM_HIDDEN_SIZE,
                               speeding.HYPERPARAMS.HIDDEN_LAYER_SIZE,
                               speeding.HYPERPARAMS.DROPOUT_PROB,
                               speeding.HYPERPARAMS.NUM_LAYERS,
                               speeding.HYPERPARAMS.TARGET_SIZE,
                               speeding.HYPERPARAMS.BATCH_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=speeding.HYPERPARAMS.LEARNING_RATE)

model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
min_loss = checkpoint['min_loss']
if CUDA:
    model.cuda()
    criterion = nn.MSELoss().cuda()

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True)
losses = {
  'training': [],
  'validation': []
}

In [7]:
for epoch in np.arange(checkpoint['epoch'], NUM_EPOCHS):
    logging.info('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    print('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    
    is_best = False

    training_loss = train(TRAINING_DATA, model, criterion, optimizer)
    logging.info('--- TRAINING LOSS: %f' % training_loss)
    print('--- TRAINING LOSS: %f' % training_loss)

#     validation_loss = validate(VALIDATION_DATA, model, criterion)
#     logging.info('--- VALIDATION LOSS: %f' % validation_loss)
#     print('--- VALIDATION LOSS: %f' % validation_loss)

    if training_loss < min_loss:
        logging.info('--- --- best model found so far: %f' % training_loss)
        print('--- --- best model found so far: %f' % training_loss)
        min_loss = training_loss
        is_best = True

    losses['training'].append(training_loss)
#     losses['validation'].append(validation_loss)

    save_checkpoint({
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'min_loss': min_loss,
          'optimizer' : optimizer.state_dict(),
      }, is_best)

    scheduler.step(training_loss)
    logging.info('-------------------------------------------------------')
    print('-------------------------------------------------------')

Epoch [1/50]
--- Parsing track medium-race_YZEY.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track medium-race_KAFP.csv
--- Parsing track easy-race_PVDY.csv
--- Parsing track medium-race_CYWR.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track medium-race_COET.csv
--- Parsing track easy-race_CDRV.csv
--- TRAINING LOSS: 1.327350
--- --- best model found so far: 1.327350
-------------------------------------------------------
Epoch [2/50]
--- Parsing track medium-race_YZEY.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track medium-race_KAFP.csv
--- Parsing track easy-race_PVDY.csv
--- Parsing track medium-race_CYWR.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track medium-race_COET.csv
--- Parsing track easy-race_CDRV.csv
--- TRAINING LOSS: 1.066817
--- --- best model found so far: 1.066817
-------------------------------------------------------
Epoch [3/50]
--- Parsing track medium-race_YZEY.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track medium-

--- Parsing track easy-race_UXES.csv
--- Parsing track medium-race_KAFP.csv
--- Parsing track easy-race_PVDY.csv
--- Parsing track medium-race_CYWR.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track medium-race_COET.csv
--- Parsing track easy-race_CDRV.csv
--- TRAINING LOSS: 0.188009
--- --- best model found so far: 0.188009
-------------------------------------------------------
Epoch [21/50]
--- Parsing track medium-race_YZEY.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track medium-race_KAFP.csv
--- Parsing track easy-race_PVDY.csv
--- Parsing track medium-race_CYWR.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track medium-race_COET.csv
--- Parsing track easy-race_CDRV.csv
--- TRAINING LOSS: 0.150383
--- --- best model found so far: 0.150383
-------------------------------------------------------
Epoch [22/50]
--- Parsing track medium-race_YZEY.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track medium-race_KAFP.csv
--- Parsing track easy-race_PVDY.csv

--- Parsing track easy-race_PVDY.csv
--- Parsing track medium-race_CYWR.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track medium-race_COET.csv
--- Parsing track easy-race_CDRV.csv
--- TRAINING LOSS: 0.080124
-------------------------------------------------------
Epoch [40/50]
--- Parsing track medium-race_YZEY.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track medium-race_KAFP.csv
--- Parsing track easy-race_PVDY.csv
--- Parsing track medium-race_CYWR.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track medium-race_COET.csv
--- Parsing track easy-race_CDRV.csv
--- TRAINING LOSS: 0.083614
-------------------------------------------------------
Epoch [41/50]
--- Parsing track medium-race_YZEY.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track medium-race_KAFP.csv
--- Parsing track easy-race_PVDY.csv
--- Parsing track medium-race_CYWR.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track medium-race_COET.csv
--- Parsing track easy-race_CDRV.csv
--- TRA

In [8]:
# dump losses to file
with open('split_checkpoints/{}/{}/losses_{}.pkl'.format(DIFFICULTY[-1], COMMAND, time.time()), 'wb') as file:
    pickle.dump(losses, file)